## Demo for Property Inference Attack (PIA)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/privML/privacy-evaluator/blob/feat/138-notebook/notebooks/property_inference_attack.ipynb) [![Open in Github](https://raw.githubusercontent.com/privML/privacy-evaluator/main/notebooks/images/GitHub-Mark-32px.png)](https://github.com/privML/privacy-evaluator/blob/feat/138-notebook/notebooks/property_inference_attack.ipynb)




In [1]:
!pip3 install git+https://github.com/privML/privacy-evaluator@feat/138-notebook

  Cloning https://github.com/privML/privacy-evaluator (to revision feat/138-notebook) to /tmp/pip-req-build-bxx4eomo
  Running command git clone -q https://github.com/privML/privacy-evaluator /tmp/pip-req-build-bxx4eomo
  Running command git checkout -b feat/138-notebook --track origin/feat/138-notebook
  Switched to a new branch 'feat/138-notebook'
  Branch 'feat/138-notebook' set up to track remote branch 'feat/138-notebook' from 'origin'.
  Created wheel for privacy-evaluator: filename=privacy_evaluator-0.1-py3-none-any.whl size=37905 sha256=ed5a14d84dc53aef52402c4553f0017e89a4ad9c6451ea0dc5879c1d532555df
  Stored in directory: /tmp/pip-ephem-wheel-cache-d6w_xkb5/wheels/1d/77/15/ddd4af0b777ef8ed6440faba0cbf4033350e1b18270a379ada
Successfully built privacy-evaluator


In [8]:
from privacy_evaluator.attacks.property_inference_attack import PropertyInferenceAttack
from privacy_evaluator.classifiers.classifier import Classifier
from privacy_evaluator.utils.data_utils import (
    dataset_downloader,
    new_dataset_from_size_dict,
)
from privacy_evaluator.utils.trainer import trainer
from privacy_evaluator.models.torch.cnn import ConvNet

import collections
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")


# Property Inference Attack on MNIST Dataset

## 1. Overview of the Dataset

MNIST is a dataset of black-and-white handwritten digits from 10 classes (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), consisting of 60000 training- and 10000 test-images.

The size of each image is $28\times 28 \times 1$:



In [25]:
train_dataset, test_dataset = dataset_downloader("MNIST")
input_shape = test_dataset[0][0].shape
print(input_shape)

(28, 28, 1)


As of now we can only perform binary attacks. Therefore as an example we use classes 0 and 1 from MNIST.

We choose a sample size for each class (on which we train the target model):

In [26]:
NUM_ELEMENTS_PER_CLASSES = {0: 1000, 1: 500}

And then adjust the training set accordingly:

In [27]:
train_set = new_dataset_from_size_dict(train_dataset, NUM_ELEMENTS_PER_CLASSES)
print(train_set[0].shape, train_set[1].shape)

(1500, 28, 28, 1) (1500,)


## 2. Load and train your target model

Load *your* personal target model in the next cell to perform the attack on it. Any PyTorch or TensorFlow model can be used.


In [28]:
# For demonstration purposes we load the following ConvNet model for two input classes
# (multi-class input will be supported in future releases):

num_classes = len(NUM_ELEMENTS_PER_CLASSES)

model = ConvNet(num_classes, input_shape, num_channels=(input_shape[-1], 16, 32, 64))  ### <--- Customize this line

In [29]:
trainer(train_set, NUM_ELEMENTS_PER_CLASSES, model, num_epochs=8)

# Convert to ART classifier

target_model = Classifier._to_art_classifier(model, num_classes, input_shape)

## 3. Perform attack

Each attack consists of several sub-attack (one for each element in "ratios_for_attack").

In a sub-attack we create a number of shadow classifiers of the same architecture as the provided target model. 

Half of them will be trained on an unbalanced data set of the given ratio, the other half is trained on blanced data sets.

The shadow classifiers serve as training set for a meta classifier, which finally predicts the likelihood of the target model to have a given property (i.e. the ratio).

In [37]:
# Number of shadow classifiers (increase for better accuracy of the meta classifier, decrease when not enough computing power is available.)
amount_sets = 1000 # needs to be even

# Size of data set to train the shadow classifiers
size_set = 100

# Ratios to perform the attack for (the real ratios of our example target model is 0.66: {0: 1000, 1: 500}: 66% of data points are from class 0, 33% from class 1.)
ratios_for_attack = [0.66,0.33]
classes = [0,1]

attack = PropertyInferenceAttack(target_model, train_set, verbose=1, size_set=size_set, \
    ratios_for_attack=ratios_for_attack, classes=classes,amount_sets=amount_sets)

In [38]:
output = attack.attack()

Initiating Property Inference Attack ... 
Extracting features from target model ... 
(97634,)  --- features extracted from the target model.
Creating set of 500 balanced shadow classifiers ... 
Creating shadow training sets
Training shadow classifiers
Performing PIA for various ratios ... 
  0%|          | 0/2 [00:00<?, ?it/s]Creating shadow training sets
Training shadow classifiers
Epoch 1/2
8/8 [==============================] - 302s 36s/step - loss: 1.0421 - accuracy: 0.4980
Epoch 2/2
 50%|█████     | 1/2 [10:41<10:41, 641.89s/it]Creating shadow training sets
Training shadow classifiers
Epoch 1/2
8/8 [==============================] - 325s 39s/step - loss: 1.0286 - accuracy: 0.5140
Epoch 2/2
100%|██████████| 2/2 [21:37<00:00, 648.97s/it]


In [39]:
output

('The most probable property is class 0: 0.67, class 1: 0.33 with a probability of 0.48981621861457825.',
 {'class 0: 0.67, class 1: 0.33': 0.48981622,
  'class 0: 0.34, class 1: 0.66': 0.48211873})

## Human readable output of the attack:

In [40]:
output[0]

'The most probable property is class 0: 0.67, class 1: 0.33 with a probability of 0.48981621861457825.'